# Synthetische data CEDA

#### To do
- Remove a small part of the VSV'ers in each concat?
- Harmonize ILT/CREBO number with corresponding description?
- Add constraint in A04 for the columns MELDING_VERZUIM_WETTELIJK and MELDING_VERZUIM_NIET_WETTELIJK. Should be onehotencoded to 'J' and 'N'

In [52]:
## Load packages 
import pandas as pd
import numpy as np
import random
import sdv
import sdv.single_table
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer
from sdv.evaluation.single_table import evaluate_quality, run_diagnostic, get_column_plot

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None) 

In [2]:
# A04 files - VSV dataset. Alle studenten in A04 zitten ook in het A05 bestand van dezelfde maand
# Deze rapportage bevat alle jongeren uit de startpopulatie die in de rapportagemaand geen relevante inschrijving meer hebben en mogelijk een VSV-er zouden kunnen zijn
# Kan dezelfde studenten per maand bevatten, maar dit hoeft niet: een student kan zich immers elders inschrijven
a04_11_22 = pd.read_csv('data/raw/202211A04I27GZ.csv', sep = ';')
a04_12_22 = pd.read_csv('data/raw/202212A04I27GZ.csv', sep = ';')
a04_01_23 = pd.read_csv('data/raw/202301A04I27GZ.csv', sep = ';')
a04_02_23 = pd.read_csv('data/raw/202302A04I27GZ.csv', sep = ';')
a04_03_23 = pd.read_csv('data/raw/202303A04I27GZ.csv', sep = ';')
a04_04_23 = pd.read_csv('data/raw/202304A04I27GZ.csv', sep = ';')
a04_05_23 = pd.read_csv('data/raw/202305A04I27GZ.csv', sep = ';')
a04_06_23 = pd.read_csv('data/raw/202306A04I27GZ.csv', sep = ';')
a04_07_23 = pd.read_csv('data/raw/202307A04I27GZ.csv', sep = ';')
a04_08_23 = pd.read_csv('data/raw/202308A04I27GZ.csv', sep = ';')
a04_09_23 = pd.read_csv('data/raw/202309A04I27GZ.csv', sep = ';')
a04_10_23 = pd.read_csv('data/raw/202310A04I27GZ.csv', sep = ';')

# A14 - VSV dataset 
# De A14 wordt aangemaakt over de maanden oktober t/m december na het rapportagejaar en bevat de mutaties die nog betrekking hebben op dit voorgaande rapportagejaar. 
# De A14 is verder identiek aan de A04. Alle VSV'ers uit A14 bestanden zitten ook in het bestand van a05_10_23. Niet per se in de oudere bestanden. 
a14_11_23 = pd.read_csv('data/raw/202311A14I27GZ.csv', sep = ';')
a14_12_23 = pd.read_csv('data/raw/202312A14I27GZ.csv', sep = ';')
a14_01_24 = pd.read_csv('data/raw/202401A14I27GZ.csv', sep = ';')

In [3]:
# A05 files - starter dataset
# Deze rapportage bevat alle jongeren die aan het begin van het rapportagejaar (1 oktober) een relevante inschrijving hebben en die bij uitval mogelijk een VSV-er zouden kunnen zijn. 
a05_11_22 = pd.read_csv('data/raw/202211A05I27GZ.csv', sep = ';')
a05_12_22 = pd.read_csv('data/raw/202212A05I27GZ.csv', sep = ';')
a05_01_23 = pd.read_csv('data/raw/202301A05I27GZ.csv', sep = ';')
a05_02_23 = pd.read_csv('data/raw/202302A05I27GZ.csv', sep = ';')
a05_03_23 = pd.read_csv('data/raw/202303A05I27GZ.csv', sep = ';')
a05_04_23 = pd.read_csv('data/raw/202304A05I27GZ.csv', sep = ';')
a05_05_23 = pd.read_csv('data/raw/202305A05I27GZ.csv', sep = ';')
a05_06_23 = pd.read_csv('data/raw/202306A05I27GZ.csv', sep = ';')
a05_07_23 = pd.read_csv('data/raw/202307A05I27GZ.csv', sep = ';')
a05_08_23 = pd.read_csv('data/raw/202308A05I27GZ.csv', sep = ';')
a05_09_23 = pd.read_csv('data/raw/202309A05I27GZ.csv', sep = ';')
a05_10_23 = pd.read_csv('data/raw/202310A05I27GZ.csv', sep = ';')

In [4]:
## NenR bestaat uit de VSV'ers van NSRP
# Voorlopig & definitief: rapportage startpopulatie - starter dataset

# De rapportage Startpopulatie (NRSP) toont de gegevens van leerlingen die aan het begin van het rapportagejaar staan ingeschreven in het Register Onderwijsdeelnemers (ROD) 
# én die bij uitval voldoen aan de definitie van voortijdig schoolverlater. 
# Het rapportagejaar start op 1 oktober van het huidige schooljaar en loopt tot 30 september van het volgende schooljaar.
# In februari/maart verschijnt de rapportage met voorlopige cijfers. In oktober/november plaatsen we de rapportage met definitieve cijfers.
nrsp_23_v = pd.read_csv('data/raw/NRSP2023VI27GZ.csv', sep = ';')
nrsp_23_d = pd.read_csv('data/raw/NRSP2023DI27GZ.csv', sep = ';')

# Voorlopig & definitief: naam en rugnummers- VSV dataset
# Met de rapportage Namen en rugnummers (NenR) kunt u controleren wie voortijdig schoolverlaters zijn binnen uw school, gemeente of regio.
# In februari/maart verschijnt de rapportage op basis van de voorlopige cijfers. In oktober/november plaatsen we de rapportage op basis van de definitieve cijfers.
# De rapportage Namen en rugnummers hoort bij de rapportage Startpopulatie.
nenr_23_v = pd.read_csv('data/raw/NenR2023VI27GZ.csv', sep = ';')
nenr_23_d = pd.read_csv('data/raw/NenR2023DI27GZ.csv', sep = ';')

# Data Exploration

In [226]:
display(a05_11_22.head(1))

,#BRIN,NAAM_INSTELLING,RAPPORTAGE_MAAND,BSN_ONDERWIJSNR,VOORNAMEN,VOORVOEGSEL,ACHTERNAAM,WOONGEMEENTE,GEBOORTEDATUM,ILT/CREBO,ILT/CREBO_OMS,LEERWEG,BRIN_NR,NAAM_INST,ONDERWIJSAANBIEDERCODE,ONDERWIJSAANBIEDER_NAAM,GEMCODE,RMC_REGIO,LOCATIECODE,ONDERWIJSLOCATIECODE,ONDERWIJSLOCATIE_POSTCODE,ONDERWIJSLOCATIE_HUISNUMMER,BRIN_VOLGNR,NIVEAU,INSCHR_VLGNR,STRAAT,HUISNUMMER,HUISLETTER,HUISNR_AAND,HUISNR_TOEV,ADRES_LOCATIE,POSTCODE,WOONPLAATS,INDICATIE_STARTKWALIFICATIE
0,27GZ,ROC Mondriaan,202210,248456349,Noël,van,Aacken,LEIDSCHENDAM-VOORBURG,24-12-2002,25103,Metselaar,BBL,27GZ,ROC Mondriaan,100A178,School voor Bouw en Infra,1916,REGIO 28,NaN,100X750,2544ED,10,NaN,Mbo2 opleidingen,3,van Heurnstraat,159.0,NaN,NaN,NaN,NaN,2274NK,VOORBURG,N


In [228]:
display(a04_11_22.head(1))

,#BRIN_NR,NAAM_INST,RAPPORTAGEMAAND,PERSOONSGEBONDENNUMMER,ACHTERNAAM,VOORVOEGSEL,ALLE_VOORNAMEN,WOONGEMEENTE,GEBOORTEDATUM,ONDERWIJSCODE,ONDERWIJSNAAM,LEERWEG,DATUM_UITSCHRIJVING,MELDING_VERZUIM_WETTELIJK,REDEN,BRIN,NAAM_INSTELLING,ONDERWIJSAANBIEDERCODE,NAAM_ONDERWIJSAANBIEDER,MELDING_VERZUIM_NIET_WETTELIJK,GEMEENTECODE,RMC_REGIO,REDEN_UITSTROOM,LOCATIECODE,ONDERWIJSLOCATIECODE,POSTCODE_ONDERWIJSLOCATIE,HUISNUMMER_ONDERWIJSLOCATIE,BRIN_VOLGNUMMER,NIVEAU,INSCHRIJVINGSVOLGNUMMER,STRAAT,HUISNUMMER,HUISLETTER,HUISNUMMER_TOEVOEGING,HUISNUMMER_AANDUIDING,ADRES_LOCATIE,POSTCODE,WOONPLAATS,VANAF_WANNEER
0,27GZ,ROC Mondriaan,202210,254355869,Aissa,Ben,Charaf Addin,'S-GRAVENHAGE,05-08-2004,25498,Helpende Zorg en Welzijn,BOL,17-10-2022,J,GEEN INSCHRIJVING,27GZ,ROC Mondriaan,100A186,School voor Zorg en Welzijn Den Haag,N,518,REGIO 28,5.0,NaN,100X746,2521DB,72,NaN,Mbo2 opleidinge,3,Laakweg,177.0,NaN,NaN,NaN,NaN,2521SG,'S-GRAVENHAGE,202210


In [229]:
display(a14_11_23.head(1))

,#BRIN_NR,NAAM_INST,RAPPORTAGEMAAND,PERSOONSGEBONDENNUMMER,ACHTERNAAM,VOORVOEGSEL,ALLE_VOORNAMEN,WOONGEMEENTE,GEBOORTEDATUM,ONDERWIJSCODE,ONDERWIJSNAAM,LEERWEG,DATUM_UITSCHRIJVING,MELDING_VERZUIM_WETTELIJK,REDEN,BRIN,NAAM_INSTELLING,ONDERWIJSAANBIEDERCODE,NAAM_ONDERWIJSAANBIEDER,MELDING_VERZUIM_NIET_WETTELIJK,GEMEENTECODE,RMC_REGIO,REDEN_UITSTROOM,LOCATIECODE,ONDERWIJSLOCATIECODE,POSTCODE_ONDERWIJSLOCATIE,HUISNUMMER_ONDERWIJSLOCATIE,BRIN_VOLGNUMMER,NIVEAU,INSCHRIJVINGSVOLGNUMMER,STRAAT,HUISNUMMER,HUISLETTER,HUISNUMMER_TOEVOEGING,HUISNUMMER_AANDUIDING,ADRES_LOCATIE,POSTCODE,WOONPLAATS,VANAF_WANNEER
0,27GZ,ROC Mondriaan,202310,254469322,Aarden,NaN,Joey Gabriël Wilhelm,'S-GRAVENHAGE,15-08-2004,25779,Persoonlijk begeleider maatschappelijke zorg,BOL,31-07-2023,J,GEEN INSCHRIJVING,27GZ,ROC Mondriaan,100A188,ROC Mondriaan Maatschappelijke zorg & verzorgende,N,518,REGIO 28,1.0,NaN,100X755,2493WN,19,NaN,Mbo4 opleidinge,2,Tuinzijde,4.0,NaN,NaN,NaN,NaN,2543XG,'S-GRAVENHAGE,202307


In [230]:
display(a04_11_22.head(1))

,#BRIN_NR,NAAM_INST,RAPPORTAGEMAAND,PERSOONSGEBONDENNUMMER,ACHTERNAAM,VOORVOEGSEL,ALLE_VOORNAMEN,WOONGEMEENTE,GEBOORTEDATUM,ONDERWIJSCODE,ONDERWIJSNAAM,LEERWEG,DATUM_UITSCHRIJVING,MELDING_VERZUIM_WETTELIJK,REDEN,BRIN,NAAM_INSTELLING,ONDERWIJSAANBIEDERCODE,NAAM_ONDERWIJSAANBIEDER,MELDING_VERZUIM_NIET_WETTELIJK,GEMEENTECODE,RMC_REGIO,REDEN_UITSTROOM,LOCATIECODE,ONDERWIJSLOCATIECODE,POSTCODE_ONDERWIJSLOCATIE,HUISNUMMER_ONDERWIJSLOCATIE,BRIN_VOLGNUMMER,NIVEAU,INSCHRIJVINGSVOLGNUMMER,STRAAT,HUISNUMMER,HUISLETTER,HUISNUMMER_TOEVOEGING,HUISNUMMER_AANDUIDING,ADRES_LOCATIE,POSTCODE,WOONPLAATS,VANAF_WANNEER
0,27GZ,ROC Mondriaan,202210,254355869,Aissa,Ben,Charaf Addin,'S-GRAVENHAGE,05-08-2004,25498,Helpende Zorg en Welzijn,BOL,17-10-2022,J,GEEN INSCHRIJVING,27GZ,ROC Mondriaan,100A186,School voor Zorg en Welzijn Den Haag,N,518,REGIO 28,5.0,NaN,100X746,2521DB,72,NaN,Mbo2 opleidinge,3,Laakweg,177.0,NaN,NaN,NaN,NaN,2521SG,'S-GRAVENHAGE,202210


In [231]:
display(nrsp_23_v.head(1))

,#BURGERSERVICENUMMER,ONDERWIJSNUMMER,GEBOORTEDATUM,GESLACHT,ACHTERNAAM,VOORVOEGSEL,VOORNAMEN,POSTCODE_CIJFERS,RMC_REGIO,RMC_REGIO_NAAM,GEMCODE,GEMEENTENAAM,BRIN,BRIN_VOLGNR,BRIN_VEST_NAAM,NIVEAU,ONDERWIJSSOORT,CREBO,CREBO_OMSCHRIJVING,ONDERWIJSAANBIEDER_CODE,ONDERWIJSAANBIEDER_NAAM,ONDERWIJSLOCATIE_CODE,ONDERWIJSLOCATIE_STRAATNAAM,ONDERWIJSLOCATIE_HUISNUMMER,ONDERWIJSLOCATIE_HUISNUMMER_TOEVOEGING,ONDERWIJSLOCATIE_POSTCODE,ONDERWIJSLOCATIE_PLAATSNAAM
0,0,176005724,20050805,M,Boutoub,NaN,Adam,2516.0,28,Haaglanden/Westland,518,'s-Gravenhage,27GZ,0,ROC Mondriaan,Mbo niveau 1,Bol1,25742,"Assistent horeca, voeding of voedingsindustrie",100A176,ROC Mondriaan Entreeopleidingen,111X981,Meppelweg,339,NaN,2544AH,'s-Gravenhage


In [232]:
display(nenr_23_v.head(1))

,#BURGERSERVICENUMMER,ONDERWIJSNUMMER,GEBOORTEDATUM,GESLACHT,ACHTERNAAM,VOORVOEGSEL,VOORNAMEN,POSTCODE_CIJFERS,RMC_REGIO,RMC_REGIO_NAAM,GEMCODE,GEMEENTENAAM,BRIN,BRIN_VOLGNR,BRIN_VEST_NAAM,CREBO,CREBO_OMSCHRIJVING,EXAMEN,IND_BEKOSTIGING,NIVEAU,ONDERWIJSSOORT,ONDERWIJSAANBIEDER_CODE,ONDERWIJSAANBIEDER_NAAM,ONDERWIJSLOCATIE_CODE,ONDERWIJSLOCATIE_STRAATNAAM,ONDERWIJSLOCATIE_HUISNUMMER,ONDERWIJSLOCATIE_HUISNUMMER_TOEVOEGING,ONDERWIJSLOCATIE_POSTCODE,ONDERWIJSLOCATIE_PLAATSNAAM
0,239365987,0,20001013,V,Koçaağa,NaN,Ilknur,2543,28,Haaglanden/Westland,518,'s-Gravenhage,27GZ,0,ROC Mondriaan,25697,Gespecialiseerd pedagogisch medewerker,NaN,J,Mbo niveau 4,Bol4,100A187,ROC Mondriaan Pedagogische en Sociale beroep,100X756,Brouwersgracht,50,NaN,2512ER,'s-Gravenhage


In [237]:
print ('Length A04 files')
print(len(a04_11_22))
print(len(a04_12_22))
print(len(a04_01_23))
print(len(a04_02_23))
print(len(a04_03_23))
print(len(a04_04_23))
print(len(a04_05_23))
print(len(a04_06_23))
print(len(a04_07_23))
print(len(a04_08_23))
print(len(a04_09_23))
print(len(a04_10_23))
print ('')
print ('Length A14 files')
print(len(a14_11_23))
print(len(a14_12_23))
print(len(a14_01_24))

Length A04 files
33
104
201
402
481
597
714
809
955
1145
1087
1239

Length A14 files
1276
1288
1297


In [12]:
print ('Length A05 files')
print (len(a05_11_22))
print (len(a05_12_22))
print (len(a05_01_23))
print (len(a05_02_23))
print (len(a05_03_23))
print (len(a05_04_23))
print (len(a05_05_23))
print (len(a05_06_23))
print (len(a05_07_23))
print (len(a05_08_23))
print (len(a05_09_23))
print (len(a05_10_23))

Length A05 files
15558
15532
15520
15517
15514
15513
15512
15512
15511
15510
15510
15509


In [13]:
print ('Length NSRP files')
print (len(nrsp_23_v))
print (len(nrsp_23_d))
print ('')
print ('Length NenR files')
print (len(nenr_23_v))
print (len(nenr_23_d))

Length NSRP files
14460
14439

Length NenR files
1103
1078


# A04, A05 & A14 files

In [ ]:
# Subset 5000 random rows from the A05 file to train the CTGANSynthesizer on. Otherwise it causes crashes or BSoDs 
subset_a05 = a05_11_22.sample(n=5000)

In [ ]:
# Create metadata and specify column types
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(subset_a05)

# Manually update the metadata for the specific column
#metadata.update_column(column_name='BSN_ONDERWIJSNR', sdtype='id')
#metadata.update_column(column_name='ILT/CREBO', sdtype='id')

# Create model with updated metadata
CTGAN = CTGANSynthesizer(
    metadata,  # Use metadata from train set
    enforce_rounding=True,  # Use same decimals as original dataset
    enforce_min_max_values=True,  # Min and max value per column are the same as in original dataset
    epochs=50, # Currently 50 epochs on a subset of 5000 rows 
    batch_size=500,
    generator_lr=0.0002,
    verbose=True
)

CTGAN.fit(subset_a05)

In [174]:
# Create synthetic dataset
synth_a05_11_22 = CTGAN.sample(num_rows=15558)

# Save output file
synth_a05_11_22.to_csv('data/output/synth_a05_11_22.csv', sep=';', index=False) 

## Feature engineering on synthetic dataset

In [65]:
# Load synthetic datafile
synthetic_a05_11_22 = pd.read_csv('data/output/synth_a05_11_22.csv', sep = ';')

In [114]:
# Store the values from the POSTCODE and ONDERWIJSLOCATIE_POSTCODE columns of a05_11_22, shuffle the zip codes randomly and then insert them
postcodes = a05_11_22['POSTCODE'].tolist()
np.random.shuffle(postcodes)
synthetic_a05_11_22['POSTCODE'] = postcodes

postcodes_loc = a05_11_22['ONDERWIJSLOCATIE_POSTCODE'].tolist()
np.random.shuffle(postcodes_loc)
synthetic_a05_11_22['ONDERWIJSLOCATIE_POSTCODE'] = postcodes_loc

In [120]:
# Function replace the random amount of digits of the 'BSN_ONDERWIJSNR' column with a random 9-digit number
def generate_unique_random_numbers(n):
    numbers = set()
    while len(numbers) < n:
        number = random.randint(100000000, 999999999)
        numbers.add(number)
    return list(numbers)

unique_random_numbers = generate_unique_random_numbers(len(synthetic_a05_11_22))
synthetic_a05_11_22['BSN_ONDERWIJSNR'] = unique_random_numbers

In [122]:
# List of the top 10 unisex names in the Netherlands
random_names = ['Noa', 'Luca', 'Sem', 'Sam', 'Isa', 'Nova', 'Bo', 'Jip', 'Lou', 'Charlie']

# Function to replace NaN values with top unisex names at random
def replace_nan_with_random_name(df, column_name, random_names):
    df[column_name] = df[column_name].apply(lambda x: random.choice(random_names) if pd.isna(x) else x)
    return df

synthetic_a05_11_22 = replace_nan_with_random_name(synthetic_a05_11_22, 'VOORNAMEN', random_names)

In [152]:
# Create synthetic A05 files based on the synthetic_a05_11_22 file while adjusting the 'RAPPORTAGE_MAAND' column for each month
synthetic_a05_12_22 = synthetic_a05_11_22.copy()
synthetic_a05_12_22['RAPPORTAGE_MAAND'] = 202211

synthetic_a05_01_23 = synthetic_a05_11_22.copy()
synthetic_a05_01_23['RAPPORTAGE_MAAND'] = 202212

synthetic_a05_02_23 = synthetic_a05_11_22.copy()
synthetic_a05_02_23['RAPPORTAGE_MAAND'] = 202301

synthetic_a05_03_23 = synthetic_a05_11_22.copy()
synthetic_a05_03_23['RAPPORTAGE_MAAND'] = 202302

synthetic_a05_04_23 = synthetic_a05_11_22.copy()
synthetic_a05_04_23['RAPPORTAGE_MAAND'] = 202303

synthetic_a05_05_23 = synthetic_a05_11_22.copy()
synthetic_a05_05_23['RAPPORTAGE_MAAND'] = 202304

synthetic_a05_06_23 = synthetic_a05_11_22.copy()
synthetic_a05_06_23['RAPPORTAGE_MAAND'] = 202305

synthetic_a05_07_23 = synthetic_a05_11_22.copy()
synthetic_a05_07_23['RAPPORTAGE_MAAND'] = 202306

synthetic_a05_08_23 = synthetic_a05_11_22.copy()
synthetic_a05_08_23['RAPPORTAGE_MAAND'] = 202307

synthetic_a05_09_23 = synthetic_a05_11_22.copy()
synthetic_a05_09_23['RAPPORTAGE_MAAND'] = 202308

synthetic_a05_10_23 = synthetic_a05_11_22.copy()
synthetic_a05_10_23['RAPPORTAGE_MAAND'] = 202309

In [154]:
# Save the A05 synthetic datasets
synthetic_a05_11_22.to_csv('data/output/202211A05I27GZ.csv', sep=';', index=False) 
synthetic_a05_12_22.to_csv('data/output/202212A05I27GZ.csv', sep=';', index=False) 
synthetic_a05_01_23.to_csv('data/output/202301A05I27GZ.csv', sep=';', index=False) 
synthetic_a05_02_23.to_csv('data/output/202302A05I27GZ.csv', sep=';', index=False) 
synthetic_a05_03_23.to_csv('data/output/202303A05I27GZ.csv', sep=';', index=False) 
synthetic_a05_04_23.to_csv('data/output/202304A05I27GZ.csv', sep=';', index=False)
synthetic_a05_05_23.to_csv('data/output/202305A05I27GZ.csv', sep=';', index=False) 
synthetic_a05_06_23.to_csv('data/output/202306A05I27GZ.csv', sep=';', index=False) 
synthetic_a05_07_23.to_csv('data/output/202307A05I27GZ.csv', sep=';', index=False) 
synthetic_a05_08_23.to_csv('data/output/202308A05I27GZ.csv', sep=';', index=False) 
synthetic_a05_09_23.to_csv('data/output/202309A05I27GZ.csv', sep=';', index=False) 
synthetic_a05_10_23.to_csv('data/output/202310A05I27GZ.csv', sep=';', index=False) 

In [155]:
# Function to convert A05 dataframe into an A04 dataframe
def create_new_month (number, dataset_input): 
    dataset = dataset_input.sample(n=number, replace=True)
    synthetic_dataset = pd.DataFrame({
        "BRIN_NR": dataset["#BRIN"],
        "NAAM_INST": dataset["NAAM_INSTELLING"],
        "RAPPORTAGEMAAND": dataset["RAPPORTAGE_MAAND"],
        "PERSOONSGEBONDENNUMMER": dataset["BSN_ONDERWIJSNR"],
        "ACHTERNAAM": dataset["ACHTERNAAM"],
        "VOORVOEGSEL": dataset["VOORVOEGSEL"],
        "ALLE_VOORNAMEN": dataset["VOORNAMEN"],
        "WOONGEMEENTE": dataset["WOONGEMEENTE"],
        "GEBOORTEDATUM": dataset["GEBOORTEDATUM"],
        "ONDERWIJSCODE": dataset["ILT/CREBO"],
        "ONDERWIJSNAAM": dataset["ILT/CREBO_OMS"],
        "LEERWEG": dataset["LEERWEG"],
        "DATUM_UITSCHRIJVING": np.nan,
        "MELDING_VERZUIM_WETTELIJK": 'J', # Might need adjustment  
        "REDEN": np.nan,
        "BRIN": dataset["BRIN_NR"],
        "NAAM_INSTELLING": dataset["NAAM_INST"],
        "ONDERWIJSAANBIEDERCODE": dataset["ONDERWIJSAANBIEDERCODE"],
        "NAAM_ONDERWIJSAANBIEDER": dataset["ONDERWIJSAANBIEDER_NAAM"],
        "MELDING_VERZUIM_NIET_WETTELIJK": 'N', # Might need adjustment   
        "GEMEENTECODE": dataset["GEMCODE"],
        "RMC_REGIO": dataset["RMC_REGIO"],
        "REDEN_UITSTROOM": np.nan,
        "LOCATIECODE": dataset["LOCATIECODE"],
        "ONDERWIJSLOCATIECODE": dataset["ONDERWIJSLOCATIECODE"],
        "POSTCODE_ONDERWIJSLOCATIE": dataset["ONDERWIJSLOCATIE_POSTCODE"],
        "HUISNUMMER_ONDERWIJSLOCATIE": dataset["ONDERWIJSLOCATIE_HUISNUMMER"],
        "BRIN_VOLGNUMMER": dataset["BRIN_VOLGNR"],
        "NIVEAU": dataset["NIVEAU"],
        "INSCHRIJVINGSVOLGNUMMER": dataset["INSCHR_VLGNR"],
        "STRAAT": dataset["STRAAT"],
        "HUISNUMMER": dataset["HUISNUMMER"],
        "HUISLETTER": dataset["HUISLETTER"],
        "HUISNUMMER_TOEVOEGING": dataset["HUISNR_TOEV"],
        "HUISNUMMER_AANDUIDING": dataset["HUISNR_AAND"],
        "ADRES_LOCATIE": dataset["ADRES_LOCATIE"],
        "POSTCODE": dataset["POSTCODE"],
        "WOONPLAATS": dataset["WOONPLAATS"],
        "VANAF_WANNEER": dataset["RAPPORTAGE_MAAND"]
})
    return synthetic_dataset

In [156]:
# Apply create_new_month on all the A05 dataframes. Sampling based on actual A04 data 
temp_a04_11_22 = create_new_month (30, synthetic_a05_11_22)
temp_a04_12_22 = create_new_month (70, synthetic_a05_12_22)
temp_a04_01_23 = create_new_month (100, synthetic_a05_01_23)
temp_a04_02_23 = create_new_month (200, synthetic_a05_02_23)
temp_a04_03_23 = create_new_month (100, synthetic_a05_03_23)
temp_a04_04_23 = create_new_month (100, synthetic_a05_04_23)
temp_a04_05_23 = create_new_month (100, synthetic_a05_05_23)
temp_a04_06_23 = create_new_month (100, synthetic_a05_06_23)
temp_a04_07_23 = create_new_month (100, synthetic_a05_07_23)
temp_a04_08_23 = create_new_month (100, synthetic_a05_08_23)
temp_a04_09_23 = create_new_month (100, synthetic_a05_09_23)
temp_a04_10_23 = create_new_month (200, synthetic_a05_10_23)

temp_a14_11_23 = create_new_month (10, synthetic_a05_10_23)
temp_a14_12_23 = create_new_month (10, synthetic_a05_10_23)
temp_a14_01_24 = create_new_month (10, synthetic_a05_10_23)

In [157]:
# Concatenate each month with the prior months and remove duplicate students based on the 'PERSOONSGEBONDENNUMMER' column  
# List of dataframes
dataframes = [temp_a04_11_22, temp_a04_12_22, temp_a04_01_23, temp_a04_02_23, temp_a04_03_23, temp_a04_04_23, temp_a04_05_23, temp_a04_06_23, temp_a04_07_23, temp_a04_08_23, temp_a04_09_23, 
              temp_a04_10_23, temp_a14_11_23, temp_a14_12_23, temp_a14_01_24]

# List of names for the synthetic dataframes
synthetic_names = ["synthetic_a04_11_22", "synthetic_a04_12_22", "synthetic_a04_01_23", "synthetic_a04_02_23", "synthetic_a04_03_23", "synthetic_a04_04_23", "synthetic_a04_05_23", "synthetic_a04_06_23", 
                   "synthetic_a04_07_23", "synthetic_a04_08_23", "synthetic_a04_09_23", "synthetic_a04_10_23", "synthetic_a14_11_23", "synthetic_a14_12_23", "synthetic_a14_01_24"]


# Initialize the first synthetic dataframe with the first dataframe
globals()[synthetic_names[0]] = dataframes[0]

# Loop through the rest of the dataframes and append them to the previous synthetic dataframe
for i in range(1, len(dataframes)):
    globals()[synthetic_names[i]] = pd.concat([globals()[synthetic_names[i-1]], dataframes[i]]).drop_duplicates(subset='PERSOONSGEBONDENNUMMER')

# Check newly created dataframes  
for name in synthetic_names: print(name)

synthetic_a04_11_22
synthetic_a04_12_22
synthetic_a04_01_23
synthetic_a04_02_23
synthetic_a04_03_23
synthetic_a04_04_23
synthetic_a04_05_23
synthetic_a04_06_23
synthetic_a04_07_23
synthetic_a04_08_23
synthetic_a04_09_23
synthetic_a04_10_23
synthetic_a14_11_23
synthetic_a14_12_23
synthetic_a14_01_24


In [158]:
# Save the A04 and A14 synthetic datasets
synthetic_a04_11_22.to_csv('data/output/202211A04I27GZ.csv', sep=';', index=False) 
synthetic_a04_12_22.to_csv('data/output/202212A04I27GZ.csv', sep=';', index=False) 
synthetic_a04_01_23.to_csv('data/output/202301A04I27GZ.csv', sep=';', index=False) 
synthetic_a04_02_23.to_csv('data/output/202302A04I27GZ.csv', sep=';', index=False) 
synthetic_a04_03_23.to_csv('data/output/202303A04I27GZ.csv', sep=';', index=False) 
synthetic_a04_04_23.to_csv('data/output/202304A04I27GZ.csv', sep=';', index=False)
synthetic_a04_05_23.to_csv('data/output/202305A04I27GZ.csv', sep=';', index=False) 
synthetic_a04_06_23.to_csv('data/output/202306A04I27GZ.csv', sep=';', index=False) 
synthetic_a04_07_23.to_csv('data/output/202307A04I27GZ.csv', sep=';', index=False) 
synthetic_a04_08_23.to_csv('data/output/202308A04I27GZ.csv', sep=';', index=False) 
synthetic_a04_09_23.to_csv('data/output/202309A04I27GZ.csv', sep=';', index=False) 
synthetic_a04_10_23.to_csv('data/output/202310A04I27GZ.csv', sep=';', index=False) 

synthetic_a14_11_23.to_csv('data/output/202311A14I27GZ.csv', sep=';', index=False) 
synthetic_a14_12_23.to_csv('data/output/202312A14I27GZ.csv', sep=';', index=False) 
synthetic_a14_01_24.to_csv('data/output/202401A14I27GZ.csv', sep=';', index=False) 

# NSRP and NenR files

In [162]:
# Function to convert A05 dataframe into an NSRP dataframe
def create_new_dataframe(dataset):
    new_df = pd.DataFrame({
        '#BURGERSERVICENUMMER': dataset['BSN_ONDERWIJSNR'],
        'ONDERWIJSNUMMER': dataset['BSN_ONDERWIJSNR'],
        'GEBOORTEDATUM': dataset['GEBOORTEDATUM'],
        'GESLACHT': np.random.choice(['M', 'V'], size=len(dataset)), 
        'ACHTERNAAM': dataset['ACHTERNAAM'],
        'VOORVOEGSEL': dataset['VOORVOEGSEL'],
        'VOORNAMEN': dataset['VOORNAMEN'],
        'POSTCODE_CIJFERS': dataset['POSTCODE'].str.slice(0, 4),
        'RMC_REGIO': dataset['RMC_REGIO'],
        'RMC_REGIO_NAAM': pd.NA, #
        'GEMCODE': dataset['GEMCODE'],
        'GEMEENTENAAM': dataset['WOONGEMEENTE'],
        'BRIN': dataset['#BRIN'],
        'BRIN_VOLGNR': dataset['BRIN_NR'],
        'BRIN_VEST_NAAM': pd.NA, #
        'NIVEAU': dataset['NIVEAU'],
        'ONDERWIJSSOORT': dataset['LEERWEG'], # Should be combination of leerweg and niveau
        'CREBO': dataset['ILT/CREBO'],
        'CREBO_OMSCHRIJVING': dataset['ILT/CREBO_OMS'],
        'ONDERWIJSAANBIEDER_CODE': dataset['ONDERWIJSAANBIEDERCODE'],
        'ONDERWIJSAANBIEDER_NAAM': dataset['ONDERWIJSAANBIEDER_NAAM'],
        'ONDERWIJSLOCATIE_CODE': dataset['LOCATIECODE'],
        'ONDERWIJSLOCATIE_STRAATNAAM': dataset['STRAAT'],
        'ONDERWIJSLOCATIE_HUISNUMMER': dataset['HUISNUMMER'],
        'ONDERWIJSLOCATIE_HUISNUMMER_TOEVOEGING': dataset['HUISNR_TOEV'],
        'ONDERWIJSLOCATIE_POSTCODE': dataset['POSTCODE'],
        'ONDERWIJSLOCATIE_PLAATSNAAM': dataset['WOONPLAATS']
    })
    return new_df

In [165]:
# Create synthetic NSRP_v dataset by using create_new_dataframe function on tbe synthetic_a05_11_22 dataset
synthetic_nsrp_v = create_new_dataframe(synthetic_a05_11_22)

# Create synthetic NenR_v dataset by sampling 1100 rows from synthetic_nsrp_v
synthetic_nenr_v = synthetic_nsrp_v.sample(n=1100)

# Create the definitive NSRP and NenR datasets by dropping 100 rows from the temporary ones 
rows_to_delete_nsrp = synthetic_nsrp_v.sample(n=100)
synthetic_nsrp_d = synthetic_nsrp_v.drop(rows_to_delete_nsrp.index)

rows_to_delete_nenr = synthetic_nenr_v.sample(n=100)
synthetic_nenr_d = synthetic_nenr_v.drop(rows_to_delete_nenr.index)

In [168]:
# Insert EXAMEN and IND_BEKOSTIGING columns at the 20th and 21st position
synthetic_nenr_v.insert(19, "EXAMEN", np.nan)
synthetic_nenr_v.insert(20, "IND_BEKOSTIGING", np.nan)
synthetic_nenr_d.insert(19, "EXAMEN", np.nan)
synthetic_nenr_d.insert(20, "IND_BEKOSTIGING", np.nan)

In [169]:
synthetic_nsrp_v.to_csv('data/output/NRSP2023VI27GZ.csv', sep=';', index=False) 
synthetic_nsrp_d.to_csv('data/output/NRSP2023DI27GZ.csv', sep=';', index=False) 
synthetic_nenr_v.to_csv('data/output/NenR2023VI27GZ.csv', sep=';', index=False) 
synthetic_nenr_d.to_csv('data/output/NenR2023DI27GZ.csv', sep=';', index=False) 